# How to Handle Missing Values
1. Delete
2. Assign Value (Median,mode)
3. Predict (Statistics)

### !!! Important
## Randomness of missing data should be taken into account

- If missing values are random we can do whatever we want
- If missing values not random and caused by a different variable , structural problems we should fix structual problems and be careful with missing values


In [15]:
from  module import load_dataset
import numpy as np 
import pandas as pd
import missingno as msno
from datetime import date

In [16]:
# Pandas View Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x:'%.3f' % x)
pd.set_option('display.width', 500)

In [2]:
df = load_dataset()

### Lecture 1
### Catching Missing Values

In [3]:
# Is there any null values
df.isnull().values.any()

True

In [4]:
# Number of null values for each column
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
df.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [17]:
# Null value percentage
(df.isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)

Cabin         77.104
Age           19.865
Embarked       0.224
PassengerId    0.000
Survived       0.000
Pclass         0.000
Name           0.000
Sex            0.000
SibSp          0.000
Parch          0.000
Ticket         0.000
Fare           0.000
dtype: float64

In [5]:
# Number of not null values for each column
df.notnull().sum()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [7]:
# Total number of null values
df.isnull().sum().sum()

866

In [21]:
# Rows that at least have one null value
# 708 rows some rows have more than one null value
print(df[df.isnull().any(axis=1)].head())
print(len(df[df.isnull().any(axis=1)]))

   PassengerId  Survived  Pclass                            Name     Sex    Age  SibSp  Parch            Ticket   Fare Cabin Embarked
0            1         0       3         Braund, Mr. Owen Harris    male 22.000      1      0         A/5 21171  7.250   NaN        S
2            3         1       3          Heikkinen, Miss. Laina  female 26.000      0      0  STON/O2. 3101282  7.925   NaN        S
4            5         0       3        Allen, Mr. William Henry    male 35.000      0      0            373450  8.050   NaN        S
5            6         0       3                Moran, Mr. James    male    NaN      0      0            330877  8.458   NaN        Q
7            8         0       3  Palsson, Master. Gosta Leonard    male  2.000      3      1            349909 21.075   NaN        S
708


In [22]:
# Without null values
df[df.notnull().all(axis=1)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB


In [24]:
# get the columns has null values
na_cols = [col for col in df.columns if df[col].isnull().sum()>0]

In [25]:
na_cols

['Age', 'Cabin', 'Embarked']

In [28]:
# getting the info about missing values which column, how many, ratio
# if you want return na cols
def missing_values_table(df, na_name=False):
    # get cols have null values
    na_cols = [col for col in df.columns if df[col].isnull().sum()>0]
    # get the number of null values
    n_miss = df[na_cols].isnull().sum().sort_values(ascending=False)
    # get the ratio of the null values
    ratio = (df[na_cols].isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)
    # merge all data on a dataframe
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end ='\n')
    # return na cols if wanted
    if na_name:
        return na_cols
    

In [31]:
missing_values_table(df, True)

          n_miss  ratio
Cabin        687 77.100
Age          177 19.870
Embarked       2  0.220


['Age', 'Cabin', 'Embarked']

### Lecture 2
### Solving Missing Value Problem

- It is more sensitive in linear methods and gradient descent based methods
- If we use tree based methods like in outliers missing values can be ignored
- Because different from the linear methods or optimization methods like gradient descent tree methods more flexible and branching out, the outliers and missing values has almost no effect.Can be ignored.
- One exception regression and because of dependent variable is numerical, may take longer to reach a conclusion, If gradient descent based optimization used with a tree method because of outliers in dependent variable optimizaton can take longer

In [33]:
## 1.Delete
# This path can be choosen when there are many observations 
df.dropna().isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [34]:
## 2. Filing with simple methods
# You can fill it with mean, median, or a fixed number
df['Age'].fillna(df['Age'].mean()).isnull().sum()
df['Age'].fillna(df['Age'].median()).isnull().sum()
df['Age'].fillna(0).isnull().sum()


0

In [35]:
# Filling with mean in numerical cols
df_fill_with_mean = df.apply(lambda x:x.fillna(x.mean()) if x.dtype != 'O' else x, axis = 0) 

In [36]:
df_fill_with_mean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [37]:
def is_null(df):
    return df.isnull().sum().sort_values(ascending=False)


In [38]:
# Categorical cols is still null
is_null(df_fill_with_mean)

Cabin          687
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

- For filling categorical values the most suitable way is mode

In [41]:
df['Embarked'].mode()[0]

'S'

In [44]:
df['Embarked'].fillna(df['Embarked'].mode()[0]).isnull().sum()

0

In [45]:
# For analyzing later you can fill with string
df['Embarked'].fillna("missing")

0            S
1            C
2            S
3            S
4            S
5            Q
6            S
7            S
8            S
9            C
10           S
11           S
12           S
13           S
14           S
15           S
16           Q
17           S
18           S
19           C
20           S
21           S
22           Q
23           S
24           S
25           S
26           C
27           S
28           Q
29           S
30           C
31           C
32           Q
33           S
34           C
35           S
36           C
37           S
38           S
39           C
40           S
41           S
42           C
43           C
44           Q
45           S
46           Q
47           Q
48           C
49           S
50           S
51           S
52           C
53           S
54           C
55           S
56           S
57           C
58           S
59           S
60           C
61     missing
62           S
63           S
64           C
65           C
66        

In [49]:
# Filling categorical cols
# If has more than 10(optional) class it is not categorical but cardinal 
df.apply(lambda x:x.fillna(x.mode()[0]) if(x.dtype == 'O' and len(x.unique())<= 10 ) else x, axis = 0) .isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [ ]:
# Cabin is different. Is the missing values relly missing or has a pattern 
# We don't know
# We filled acceptable categorical variable 

- How to fill numerical variables more sensetivly?